In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [2]:
tk = yf.Ticker('TSLA')
# Expiration dates
exps = tk.options

In [3]:
tk.option_chain('2022-06-10')

ValueError: Expiration `2022-06-10` cannot be found. Available expiration are: [2022-07-08, 2022-07-15, 2022-07-22, 2022-07-29, 2022-08-05, 2022-08-19, 2022-09-16, 2022-10-21, 2022-11-18, 2022-12-16, 2023-01-20, 2023-03-17, 2023-06-16, 2023-09-15, 2024-01-19, 2024-03-15, 2024-06-21]

In [24]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

def options_chain(symbol):

    tk = yf.Ticker(symbol)
    # Expiration dates
    exps = tk.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = tk.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate'])
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
#     options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [25]:
%%time
option = options_chain('TSLA')

Wall time: 4.42 s


In [29]:
option[(option.expirationDate=='2022-06-10') & (option.strike==700)]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expirationDate,dte,CALL,mark
48,TSLA220610C00700000,2022-06-03 19:59:58+00:00,700.0,30.82,30.5,31.0,-54.239998,-63.766754,7986.0,2611.0,0.883546,True,REGULAR,USD,2022-06-10,0.010959,True,30.75
225,TSLA220610P00700000,2022-06-03 19:59:59+00:00,700.0,27.43,27.3,27.8,18.930000,222.705890,32993.0,3511.0,0.894288,False,REGULAR,USD,2022-06-10,0.010959,False,27.55
